# **Configuração Geral**

## Imports

In [87]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from fractions import Fraction

## Funções

In [88]:
# Converte uma string que representa polegadas para um float
def polegadas(value):

    # Divide a string recebida
    value = value.strip()

    # Se a string tem mais de uma parte, obtém a parte inteira e calcula a parte decimal
    if ' ' in value:
        inteiro, decimal = value.split(' ') # a parte inteira é o que está antes do ' ', a parte decimal é o que está depois
        polegadas = float(inteiro) + float(Fraction(decimal))

    else:
        polegadas = float(value)
    
    return polegadas

# -------------------------------


# **Carregando Dados**

In [89]:
# Carregando dados csv
df = pd.read_csv('data/dados_pocos.csv')
df.head()

,Unnamed: 0,Nome do Poço,Fase,Tipo Poço,LDA (Lâmina D'Água),Diâmetro Fase,Metragem,NFases
0,0,6C5G4M,1,VERTICAL,598.0,26,607.0,4
1,1,6C5G4M,2,VERTICAL,598.0,17 1/2,1306.0,4
2,2,6C5G4M,3,VERTICAL,598.0,12 1/4,1201.0,4
3,3,6C5G4M,4,VERTICAL,598.0,8 1/2,1445.0,4
4,4,8030JC,1,VERTICAL,0.0,17 1/2,410.0,3


# **Limpeza dos Dados**
- Tratamento de valores ausentes
- Remoção de outliers
- Correção de tipos de dados
- Normalização e padronização dos dados

In [90]:
# Renomear as colunas para o padrão PEP8
df.columns = ['index', 'nome_do_poco', 'fase', 'tipo_poco', 'lda_lamina_d_agua', 'diametro_fase', 'metragem', 'nfases']

# Remover a coluna de índices 'Unnamed: 0'
df = df.drop(columns=['index'])

# Exibir df
df.head()

,nome_do_poco,fase,tipo_poco,lda_lamina_d_agua,diametro_fase,metragem,nfases
0,6C5G4M,1,VERTICAL,598.0,26,607.0,4
1,6C5G4M,2,VERTICAL,598.0,17 1/2,1306.0,4
2,6C5G4M,3,VERTICAL,598.0,12 1/4,1201.0,4
3,6C5G4M,4,VERTICAL,598.0,8 1/2,1445.0,4
4,8030JC,1,VERTICAL,0.0,17 1/2,410.0,3


In [91]:
# Verificar tipos de cada coluna
print(df.dtypes)

nome_do_poco          object
fase                   int64
tipo_poco             object
lda_lamina_d_agua    float64
diametro_fase         object
metragem             float64
nfases                 int64
dtype: object


In [92]:
# Verificar quantidade de valores ausentes por coluna
missing_values = df.isna().sum()
missing_values_percentage = (missing_values / len(df)) * 100
missing_values_df = pd.DataFrame({'valores_nulos': missing_values, 'percentual': missing_values_percentage})
print(missing_values_df)

                   valores_nulos  percentual
nome_do_poco                   0         0.0
fase                           0         0.0
tipo_poco                      0         0.0
lda_lamina_d_agua              0         0.0
diametro_fase                  0         0.0
metragem                       0         0.0
nfases                         0         0.0


In [93]:
# Verificar quantidade de linhas duplicadas
duplicate_count = df.duplicated().sum()
print(f'Quantidade de linhas duplicadas: {duplicate_count}')

Quantidade de linhas duplicadas: 0


In [94]:
# Converte coluna de string para float
df['diametro_fase_polegadas'] = df['diametro_fase'].apply(polegadas)
df.drop(columns='diametro_fase', inplace=True)
df.dtypes

nome_do_poco                object
fase                         int64
tipo_poco                   object
lda_lamina_d_agua          float64
metragem                   float64
nfases                       int64
diametro_fase_polegadas    float64
dtype: object

## Corrigindo a coluna 'tipo_poco'
- O exercício informa que existem apenas 02 tipos: VERTICAL e HORIZONTAL
- Mas o dataset tem mais valores além desses: DIRECIONAL e '0'

In [95]:
df['tipo_poco'].unique()

array(['VERTICAL', 'DIRECIONAL', '0', 'HORIZONTAL'], dtype=object)

Diante da quantidade de valores incorretos, tenho que pensar numa abordagem cautelosa para substituir esses valores.

In [104]:
# Contando quantas linhas têm valores incorretos para 'tipo_poco'
counts = df['tipo_poco'].value_counts()
direcional_count = counts.get('DIRECIONAL', 0)
zero_count = counts.get('0', 0)

# Calcular a porcentagem
total_count = len(df['tipo_poco'])
direcional_perc = (direcional_count / total_count) * 100
zero_perc = (zero_count / total_count) * 100

print(f"Porcentagem de valores com DIRECIONAL: {direcional_perc:.2f}%")
print(f"Porcentagem de valores com 0: {zero_perc:.2f}%")

Porcentagem de valores com DIRECIONAL: 35.80%
Porcentagem de valores com 0: 1.42%


In [59]:
# Gravar dataframe limpo
df.to_csv('data/df_cleaned.csv', index=False)
print('Dataframe limpo gravado com sucesso!')

Dataframe limpo gravado com sucesso!


# **Análise Exploratória de Dados (EDA)**
- Extrair insights de negócio
- Estatísticas descritivas
- Distribuições de variáveis
- Análise de correlação
- Visualizações (histogramas, scatter plots, heatmaps, etc.)

In [60]:
# Carregando dataset limpo
df = pd.read_csv('data/df_cleaned.csv')
df.head()

,nome_do_poco,fase,tipo_poco,lda_lamina_d_agua,metragem,nfases,diametro_fase_polegadas
0,6C5G4M,1,VERTICAL,598.0,607.0,4,26.00
1,6C5G4M,2,VERTICAL,598.0,1306.0,4,17.50
2,6C5G4M,3,VERTICAL,598.0,1201.0,4,12.25
3,6C5G4M,4,VERTICAL,598.0,1445.0,4,8.50
4,8030JC,1,VERTICAL,0.0,410.0,3,17.50


In [61]:
# Informações gerais do dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10649 entries, 0 to 10648
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   nome_do_poco             10649 non-null  object 
 1   fase                     10649 non-null  int64  
 2   tipo_poco                10649 non-null  object 
 3   lda_lamina_d_agua        10649 non-null  float64
 4   metragem                 10649 non-null  float64
 5   nfases                   10649 non-null  int64  
 6   diametro_fase_polegadas  10649 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 582.5+ KB


In [63]:
# Estatística descritiva (apenas das colunas numéricas)
df.describe().round(2)

,fase,lda_lamina_d_agua,metragem,nfases,diametro_fase_polegadas
count,10649.00,10649.00,10649.00,10649.00,10649.00
mean,2.67,1105.84,678.04,4.30,16.84
std,1.54,733.54,564.20,1.99,8.57
min,1.00,0.00,0.00,1.00,0.00
25%,1.00,444.00,178.00,3.00,9.50
50%,2.00,1091.00,604.00,4.00,14.00
75%,4.00,1730.00,1026.00,5.00,22.00
max,8.00,2988.00,4040.00,22.00,42.00


# **Feature Engineering**
- Criação de novas variáveis
- Seleção de variáveis
- Transformações de variáveis
- Codificação de variáveis categóricas

# **Pré-processamento dos Dados**
- Divisão dos dados em treino e teste
- Escalonamento de características
- Preparação de dados para modelagem

# **Modelagem** (Machine Learning)
- Escolha dos modelos
- Treinamento dos modelos
- Avaliação de desempenho dos modelos
- Ajuste de hiperparâmetros

# **Avaliação do Modelo**
- Métricas de avaliação (precision, recall, F1-score, etc.)
- Curvas ROC e AUC
- Análise de erros

# **Deploy do Modelo**
- Gravação do modelo treinado
- Carregamento do modelo para previsões futuras

# **Conclusões**
- Sumário dos resultados
- Recomendações
- Próximos passos

# **Referências**
- Citações e referências utilizadas